<a href="https://colab.research.google.com/github/unverciftci/self-conscious-agent/blob/main/Self_Conscious_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Self-Conscious Research Agent Demo
# Run this in Google Colab with Qwen-0.5B

# !pip install transformers torch accelerate sentencepiece

import json
import time
import random
from datetime import datetime
from typing import Dict, List, Optional
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class SelfConsciousAgent:
    """A self-aware research agent that monitors its own state and performance"""

    def __init__(self, model_name="Qwen/Qwen2.5-0.5B-Instruct"):
        """Initialize the self-conscious agent with Qwen model"""
        print("🤖 Initializing Self-Conscious Agent...")

        # Load model and tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )

        # Initialize self-state
        self.self_state = {
            "identity": {
                "agent_id": "research_agent_001",
                "version": "demo_1.0",
                "model": model_name,
                "capabilities": ["text_analysis", "summarization", "question_answering"]
            },
            "current_task": {
                "type": None,
                "description": None,
                "status": "idle",
                "started": None,
                "progress": 0.0,
                "confidence": 1.0,
                "attempts": 0
            },
            "performance": {
                "tasks_completed": 0,
                "tasks_failed": 0,
                "avg_confidence": 1.0,
                "errors": [],
                "last_success": None
            },
            "resources": {
                "model_loaded": True,
                "memory_ok": True,
                "response_time": 0
            },
            "action_history": []
        }

        print(f"✅ Agent initialized: {self.self_state['identity']['agent_id']}")
        self._update_state("status", "ready")

    def _update_state(self, key: str, value, category: str = "current_task"):
        """Update self-state and trigger introspection"""
        self.self_state[category][key] = value

    def _introspect(self) -> Dict:
        """Self-monitoring: Analyze current state and determine if help is needed"""
        introspection = {
            "timestamp": datetime.now().isoformat(),
            "self_assessment": {
                "what_am_i_doing": self.self_state["current_task"]["description"],
                "current_status": self.self_state["current_task"]["status"],
                "progress": self.self_state["current_task"]["progress"],
                "confidence": self.self_state["current_task"]["confidence"],
                "performance_ratio": self._calculate_success_rate()
            },
            "needs_help": self.self_state["current_task"]["confidence"] < 0.5,
            "issues_detected": []
        }

        # Check for issues
        if self.self_state["current_task"]["attempts"] > 3:
            introspection["issues_detected"].append("Multiple attempts without success")

        if self.self_state["current_task"]["confidence"] < 0.3:
            introspection["issues_detected"].append("Very low confidence in current approach")

        if len(self.self_state["performance"]["errors"]) > 5:
            introspection["issues_detected"].append("High error rate detected")

        return introspection

    def _calculate_success_rate(self) -> float:
        """Calculate success rate from history"""
        total = self.self_state["performance"]["tasks_completed"] + \
                self.self_state["performance"]["tasks_failed"]
        if total == 0:
            return 1.0
        return self.self_state["performance"]["tasks_completed"] / total

    def _generate_response(self, prompt: str, max_length: int = 200) -> str:
        """Generate response using Qwen model"""
        try:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device)

            start_time = time.time()
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=0.7,
                do_sample=True,
                top_p=0.9
            )
            response_time = time.time() - start_time

            self._update_state("response_time", response_time, "resources")

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            # Extract only the generated part
            response = response[len(prompt):].strip()

            return response
        except Exception as e:
            self.self_state["performance"]["errors"].append(str(e))
            return f"Error in generation: {str(e)}"

    def request_human_feedback(self, reason: str) -> str:
        """Request help from human when confidence is low"""
        print("\n🆘 === HUMAN FEEDBACK REQUEST ===")
        print(f"Reason: {reason}")
        print(f"Current task: {self.self_state['current_task']['description']}")
        print(f"Confidence: {self.self_state['current_task']['confidence']:.2f}")
        print(f"Issues: {', '.join(self._introspect()['issues_detected'])}")
        print("================================\n")

        # In a real system, this would wait for human input
        # For demo, we'll simulate it
        return "Simulated feedback: Try breaking down the task into smaller steps"

    def execute_task(self, task_type: str, task_description: str, content: str) -> Dict:
        """Execute a research task with self-monitoring"""

        # Initialize task in self-state
        self._update_state("type", task_type)
        self._update_state("description", task_description)
        self._update_state("status", "starting")
        self._update_state("started", datetime.now().isoformat())
        self._update_state("progress", 0.0)
        self._update_state("attempts", self.self_state["current_task"]["attempts"] + 1)

        print(f"\n🔍 Starting task: {task_description}")

        # Self-check before starting
        pre_check = self._introspect()
        print(f"📊 Pre-task introspection: Confidence={pre_check['self_assessment']['confidence']:.2f}")

        # Simulate progressive task execution
        results = []
        confidence_scores = []

        try:
            # Step 1: Understanding the task
            self._update_state("status", "analyzing_requirements")
            self._update_state("progress", 0.2)

            understanding_prompt = f"""As a research agent, I need to {task_type} the following:
Task: {task_description}
Content: {content[:500]}...

First, I'll identify the key requirements:"""

            understanding = self._generate_response(understanding_prompt)
            results.append({"step": "understanding", "output": understanding})

            # Simulate confidence based on response quality
            confidence = 0.8 if len(understanding) > 50 else 0.4
            confidence_scores.append(confidence)
            self._update_state("confidence", confidence)

            # Check if we need help
            mid_check = self._introspect()
            if mid_check["needs_help"]:
                feedback = self.request_human_feedback("Low confidence in task understanding")
                results.append({"step": "human_feedback", "output": feedback})

            # Step 2: Processing
            self._update_state("status", "processing")
            self._update_state("progress", 0.6)

            if task_type == "summarize":
                process_prompt = f"Summarize the following in 3 key points:\n{content}"
            elif task_type == "analyze":
                process_prompt = f"Analyze the following and identify patterns:\n{content}"
            else:
                process_prompt = f"Answer based on:\n{content}\nQuestion: {task_description}"

            main_result = self._generate_response(process_prompt)
            results.append({"step": "main_processing", "output": main_result})

            # Update confidence based on result
            confidence = 0.9 if len(main_result) > 100 else 0.5
            confidence_scores.append(confidence)
            self._update_state("confidence", sum(confidence_scores) / len(confidence_scores))

            # Step 3: Validation
            self._update_state("status", "validating")
            self._update_state("progress", 0.9)

            validation_prompt = f"Check if this {task_type} result is complete and accurate:\n{main_result[:200]}"
            validation = self._generate_response(validation_prompt, max_length=100)

            # Final confidence
            final_confidence = sum(confidence_scores) / len(confidence_scores)
            self._update_state("confidence", final_confidence)

            # Complete task
            self._update_state("status", "completed")
            self._update_state("progress", 1.0)
            self.self_state["performance"]["tasks_completed"] += 1
            self.self_state["performance"]["last_success"] = datetime.now().isoformat()

            # Record in action history
            self.self_state["action_history"].append({
                "task": task_description,
                "type": task_type,
                "confidence": final_confidence,
                "timestamp": datetime.now().isoformat(),
                "success": True
            })

            # Final introspection
            final_check = self._introspect()

            return {
                "success": True,
                "task_type": task_type,
                "results": results,
                "main_output": main_result,
                "confidence": final_confidence,
                "introspection": final_check,
                "self_state_summary": {
                    "tasks_completed": self.self_state["performance"]["tasks_completed"],
                    "success_rate": self._calculate_success_rate(),
                    "current_status": self.self_state["current_task"]["status"]
                }
            }

        except Exception as e:
            # Handle failure with self-awareness
            self._update_state("status", "failed")
            self.self_state["performance"]["tasks_failed"] += 1
            self.self_state["performance"]["errors"].append(str(e))

            return {
                "success": False,
                "error": str(e),
                "introspection": self._introspect(),
                "advice": "Need to review approach or request human assistance"
            }

    def show_self_awareness(self):
        """Display current self-awareness state"""
        print("\n🧠 === SELF-AWARENESS REPORT ===")
        print(f"Identity: {self.self_state['identity']['agent_id']}")
        print(f"Current Status: {self.self_state['current_task']['status']}")
        print(f"Progress: {self.self_state['current_task']['progress']*100:.1f}%")
        print(f"Confidence: {self.self_state['current_task']['confidence']:.2f}")
        print(f"Tasks Completed: {self.self_state['performance']['tasks_completed']}")
        print(f"Success Rate: {self._calculate_success_rate()*100:.1f}%")
        print(f"Recent Errors: {len(self.self_state['performance']['errors'])}")

        introspection = self._introspect()
        if introspection['needs_help']:
            print("⚠️ HELP NEEDED: " + ", ".join(introspection['issues_detected']))

        print("================================\n")

        return self.self_state

# === DEMO USAGE ===

def run_demo():
    """Run the self-conscious agent demo"""

    print("="*50)
    print("SELF-CONSCIOUS RESEARCH AGENT DEMO")
    print("="*50)

    # Initialize agent
    agent = SelfConsciousAgent()

    # Show initial self-awareness
    agent.show_self_awareness()

    # Test samples
    test_samples = [
        {
            "type": "summarize",
            "description": "Summarize key findings about protein folding",
            "content": "Protein folding is the physical process by which a protein chain acquires its native 3-dimensional structure, a conformation that is usually biologically functional, in an expeditious and reproducible manner. It is the physical process by which a polypeptide folds into its characteristic and functional three-dimensional structure from a random coil."
        },
        {
            "type": "analyze",
            "description": "Analyze the relationship between temperature and reaction rate",
            "content": "In chemical kinetics, increasing temperature typically increases reaction rates. The Arrhenius equation shows that rate constants increase exponentially with temperature. This is because higher temperatures provide more kinetic energy to molecules, increasing the frequency and energy of collisions."
        },
        {
            "type": "question",
            "description": "What are the main challenges in drug discovery?",
            "content": "Drug discovery faces challenges including: high costs (average $1-3 billion per drug), long development times (10-15 years), high failure rates (only 1 in 5000 compounds reach market), and difficulty predicting human responses from preclinical models."
        }
    ]

    # Execute tasks with self-monitoring
    for i, sample in enumerate(test_samples, 1):
        print(f"\n{'='*50}")
        print(f"TASK {i}/{len(test_samples)}")
        print(f"{'='*50}")

        result = agent.execute_task(
            task_type=sample["type"],
            task_description=sample["description"],
            content=sample["content"]
        )

        # Show results
        print(f"\n✅ Task Result:")
        print(f"Success: {result['success']}")
        print(f"Confidence: {result.get('confidence', 0):.2f}")
        print(f"Output: {result.get('main_output', 'N/A')[:300]}...")

        # Show self-awareness after task
        time.sleep(1)  # Brief pause for readability
        agent.show_self_awareness()

    # Final report
    print("\n" + "="*50)
    print("FINAL SELF-ASSESSMENT")
    print("="*50)
    final_state = agent.show_self_awareness()

    # Save state to JSON
    with open('agent_self_state.json', 'w') as f:
        json.dump(final_state, f, indent=2, default=str)
    print("\n📁 Self-state saved to agent_self_state.json")

# Run the demo
if __name__ == "__main__":
    run_demo()